<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%201/1_9_2_Kalifornien_Hauspreise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os # u.a. zur Entwicklugn plattformübergreifender Systempfade
import pandas as pd # Datenmanagement
import numpy as np # Hilfsfunktionen für mathematische Operationen

# Datenvisualisierung
import seaborn as sns 
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split # Datensplits
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn import metrics # Modellevaluierung

In [ ]:
import os
import tarfile
import urllib.request
import shutil
import requests

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.getcwd()
CALIFORNIA_URL = "https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/california.png"
CALIFORNIA_PATH = "california.png"
FILE_PATH = "housing.csv"
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

response = requests.get(CALIFORNIA_URL, stream=True)
with open(CALIFORNIA_PATH, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
del response

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
fetch_housing_data()

df = pd.read_csv(FILE_PATH) # Wir lesen die Datei housing.csv ein

df = df.dropna() # löscht alle Zeile mit fehlenden Attributen
df = df.reset_index(drop=True) # zählt unsere Daten neu durch

description = df.describe()

bins = [0] + list(description["median_house_value"][
    ["25%", "50%", "75%"]
].astype(int)) + [np.inf]

df["house_cat"] = pd.cut(
    df["median_house_value"],
    bins=bins,
    labels=["0 - 25%", "25 - 50%", "50 - 75%", "75 - 100%"]
)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
for train_index, test_index in split.split(df, df["house_cat"]):
    df_train = df.loc[train_index]
    df_test = df.loc[test_index]
    
df_train = df_train.drop("house_cat", axis=1)
df_test = df_test.drop("house_cat", axis=1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def plot_df_on_california(df, california_path):

    ax = df.plot(
        kind="scatter", 
        x="longitude", 
        y="latitude", 
        figsize=(10, 8),
        s=df['population']/100, 
        label="Population",
        c="median_house_value", 
        cmap=plt.get_cmap("jet"),
        colorbar=True, 
        alpha=0.4
    )

    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    california_img = mpimg.imread(california_path)
    plt.imshow(
        california_img, 
        extent=[-124.85, -113.8, 32.08, 42.42], 
        alpha=0.4,
        cmap=plt.get_cmap("jet")
    )

    plt.legend();

# 1.9 Hauspreise in Kalifornien

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/cover_housing.jpg">

### Beginn der Attributsanalyse

Wir wollen nun verstehen, welche Zusammenhänge in den Eingabedaten zu unserer Zielvariable besteht. Da wir `longitude` (Längengrad) und `latitude` (Breitengrad) in unseren Daten haben, können wir diese abbilden:

In [ ]:
df_train.plot(kind="scatter", x="longitude", y="latitude", figsize=(12, 8));

Das hilft uns aber noch nicht sonderlich weiter. Wir können unsere einzelnen Punkte grafisch besser darstellen, indem wir etwa Population und Hauswerte mit aufnehmen:

In [ ]:
df_train.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, 
    s=df_train["population"]/100, label="population", figsize=(12, 8), c="median_house_value", 
    cmap=plt.get_cmap("jet"), colorbar=True, sharex=False
)
plt.legend();

Wir können die Grafik mit einem Bild hinterlegen. Hierzu greifen wir auf eine Funktion zu, die wir in einem Skript festgehalten haben und nun importieren:

In [ ]:
plot_df_on_california(df_train, CALIFORNIA_PATH)

Wir sehen ganz deutlich:
- an der Küste sind die Preise höher, im Inland sind die Preise wesentlich geringer
- teure Wohngebiete treten selten alleine auf, sondern gruppieren sich

Wir können aber noch tiefer in die Analyse gehen. Schauen wir uns dazu `ocean_proximity` (Nähe zum Meer) an, da dies wohl ein relevantes Attribut für unsere Prognose ist:

In [ ]:
ocean_proximity_value_counts = df_train["ocean_proximity"].value_counts()
ocean_proximity_value_counts

Wir haben dort fünf Ausprägungen, von denen eine Ausprägung ein Ausreißer ist (`ISLAND` mit 5 Punkten). Schauen wir uns zu diesen einmal die Median-Hauspreise an:

In [ ]:
median_house_values_by_proximity = df_train.groupby("ocean_proximity")["median_house_value"].median()
median_house_values_by_proximity

Es scheint ganz klare Unterschiede in den Preisen für dieses Attribut zu geben; schauen wir uns das einmal genauer an, indem wir nur noch Teile unserer Daten visualisieren. Wir schreiben dafür eine Hilfsfunktion:

In [ ]:
def filter_df_by_proximity(df, proximity):
    return df.loc[df["ocean_proximity"] == proximity]

Nun gehen wir schrittweise die Visualisierungen durch. Fangen wir mit `INLAND` an:

In [ ]:
plot_df_on_california(filter_df_by_proximity(df_train, "INLAND"), CALIFORNIA_PATH)

Inlandpreise sind scheinbar eher gering; schauen wir auf `1H OCEAN`:

In [ ]:
plot_df_on_california(filter_df_by_proximity(df_train, "<1H OCEAN"), CALIFORNIA_PATH)

Die Preise sind schon erkennbar höher. Blicken wir auf `NEAR OCEAN`:

In [ ]:
plot_df_on_california(filter_df_by_proximity(df_train, "NEAR OCEAN"), CALIFORNIA_PATH)

Wir stellen klar fest: Je näher die Gegend zum Meer, desto teurer. Schauen wir uns die zwei Ausstehenden an:

In [ ]:
plot_df_on_california(filter_df_by_proximity(df_train, "NEAR BAY"), CALIFORNIA_PATH)

`NEAR BAY` scheint eine klare Gegend zu repräsentieren. `ISLAND` kennen wir schon als Ausreißer:

In [ ]:
plot_df_on_california(filter_df_by_proximity(df_train, "ISLAND"), CALIFORNIA_PATH)

Für `ISLAND` haben wir nahezu keine verlässliche Datenbasis. Sollten wir hierfür überhaupt Prognosen zulassen?

Für das weitere Projekt ignorieren wir `ISLAND`. In der Praxis wäre dies jetzt jedoch ein guter Punkt zu überlegen, ob für solch seltenen Attribute Prognosen erstellt werden sollten, oder eben nicht. Eine Alternative wäre ein Hybrid: Für `INLAND`, `NEAR OCEAN` etc. darf eine Prognose getan werden, für `ISLAND` muss ein Mensch gefragt werden.

In [ ]:
df_train = df_train.drop(filter_df_by_proximity(df_train, "ISLAND").index)
df_test = df_test.drop(filter_df_by_proximity(df_test, "ISLAND").index)

Mit dem erlangten Wissen könnten wir bereits jetzt eine ganz einfache, regelbasierte Prognose erstellen. Diese hat noch nichts mit Machine Learning zu tun, wäre aber in der Lage, Prognosen zu erstellen:

In [ ]:
def predict_by_ocean_proximity(df, median_house_values_by_proximity):
    df["ocean_proximity_prediction"] = df["ocean_proximity"].apply(
        lambda x: median_house_values_by_proximity[x]
    )
    return df

Schauen wir uns das einfach einmal an:

In [ ]:
df_train = predict_by_ocean_proximity(df_train, median_house_values_by_proximity)
df_test = predict_by_ocean_proximity(df_test, median_house_values_by_proximity)
df_train.head()

Ok, wir sehen direkt, dass dieses Attribut wohl nicht ausreicht. Nichtsdestotrotz haben wir nun eine Baseline, d.h. ein Wert mit dem wir uns vergleichen können. Wir fangen klein an, und steigern uns immer weiter - eine Möglichkeit wäre etwa, als nächstes ein Machine Learning Modell zu verwenden, oder weitere Attribute zu analysieren und in die Rechnung einzubringen.

Werfen wir nun noch einen Blick auf die Korrelation der Attribute, bevor wir uns an das Machine Learning Training geben. Wir nutzen den Pearson Korrelationskoeffizienten, welcher uns - bei linearem Zusammenhang - eine Angabe darüber gibt, wie eine Variable y von der Variable x abhängt. Wir können mit `pandas` einen Blick darauf werfen:

In [ ]:
df_train.corr()["median_house_value"].abs().sort_values(ascending=False)

Das Einkommen der Bewohner eines Gebiets hat einen hohen Einfluss auf den Wert der Wohngegend - oder besser gesagt: Wenn Menschen ein gutes Gehalt verdienen, können Sie sich eher eine teure Wohnung leisten. Wir können uns diesen Sachverhalt wieder visualisieren:

In [ ]:
df_train.plot(
    kind="scatter",
    x="median_income",
    y="median_house_value",
    alpha=0.1
);

Wie schon zuvor erkannt scheint ein oberes Limit von 500.000€ für Wohngebiete in unseren Daten aufzutauchen. Wohngebiete, die teurer sind, werden auf 500.000€ gesetzt. Dies kann etwa durch die Erhebung der Daten geschehen sein. Wir könnten nun überlegen, ob wir alle Daten mit 500.000€ aus unserem Datensatz entfernen, da sich hier viele fehlerhafte Daten befinden werden (Daten, mit einem eigentlich höheren Wert). Wir entscheiden uns nun einmal dafür, diese Daten zu behalten.

Wir haben nun noch eine Idee, ob wir nicht weitere Informationen aus unseren bestehenden Daten extrahieren können, die für ein Modell vielleicht noch interessant wären. Eine ganz einfache Möglichkeit wäre hier etwa, Attribute direkt über eine Division zueinander in Relation zu stellen:

In [ ]:
def engineer_features(df):
    df["ratio_bedrooms"] = df["total_bedrooms"] / df["total_rooms"]
    df["people_per_household"] = df["population"] / df["households"]
    return df

Wir wenden diese Funktion auf unsere Daten an:

In [ ]:
df_train = engineer_features(df_train)
df_test = engineer_features(df_test)

Und können feststellen, dass diese Attribute - rein aus Betrachtung der Korrelation - für uns interessant sein könnten.

In [ ]:
df_train.corr()["median_house_value"].abs().sort_values(ascending=False)